# Agenda

1. Recap + Q&A from yesterday
2. Exercise
3. Functions
    - What are they?
    - How do we define functions?
    - Arguments and parameters
    - Return values
    - Local vs. global variables
4. Modules + packages
    - What are modules/packages?
    - How can we use them?
    - Python's standard library
    - PyPI (the Python Package Index)
    - Installing/using packages with `pip`


# Recap

1. Dictionaries
    - Dicts are key-value stores
    - There are some rules for a dict
        - Every key must be immutable
        - Keys in a dict are unique
        - Every key has a value, and every value has a key
        - There are *no* restrictions on values; they can repeat, and can be any value we want or imagine
    - We retrieve via `[]`, indicating the key whose value we want
    - We can check if a key is in a dict with `in`
        - We cannot use `in` to find if a value is in a dict
    - We can assign to a dict using `[]`
        - If the key is new, then we add a key-value pair
        - If the key exists, then we update the value for that key
    - We can iterate over a dict
        - If we iterate over the dict object, we get the keys
        - The `dict.items` method returns `(key, value)` 2-element tuples, one at a time, when we iterate. This is my favorite way to iterate over a dict
    - Three paradigms for using dicts
        - Dicts are always mutable -- but these are three conventions for working with them, that I've seen a lot of
        - Define it, retrieve from it, but never update it -- use as a small in-memory database
        - Define a dict with keys and initial values; we never add/remove keys, but we do update the values to count things
        - Define an empty dict, adding keys (as needed) and values (as needed)

2. Files
    - To work with a file, we need to use `open`, which requests help and a file object from the OS
        - When we open a file, we can specify the "mode" as the second argument to `open`, after the filename
            - `'r'` (reading, the default)
            - `'w'` (writing, which removes any previous data / zeroes out the file we open if it exists)
            - `'a'` (append, like writing, but adds to the end of a file)
    - We can read from a file in at least three ways:
        - Invoke `read()`, getting the contents, but this is considered a bit dangerous
        - Invoke `read(n)`, which returns the next `n` characters, but this is annoying, because it doesn't stop at lines.
        - Iterate over the file object, giving us one line at a time
            - Each iteration returns a string, one line in the file
            - Each line ends with `'\n'`, the line-ending character
            - When we get to the end of the file, the loop stops
    - If we want to write to a file, we use the `write` method
        - This doesn't automatically add `'\n'` to the end
    - The problem with writing is that you really need to flush + close the file, if you want to know precisely when the data is written to disk
        - You can invoke one or both of these yourself
            - `f.flush()` or `f.close()` will do these
            - You can retrieve `f.closed`, which is a `True` or `False` value, indicating if the file was closed.
        - It's common to use `with` to open a file-handling section of your code, and it automatically flushes + closes the file at the end of the block.

In [1]:
f = open('/etc/passwd')

In [2]:
f.closed

False

In [3]:
f.close()

In [4]:
f.closed

True

In [5]:
!ls *.txt

claire.txt	      mini-access-log.txt  reuven-file.txt  wcfile.txt
linux-etc-passwd.txt  nums.txt		   shoe-data.txt


# Exercise: Config writing and reading

1. Define a (small) dict with some keys and values, between 3-5 pairs.
2. Write this dict to disk in a "config file" format, meaning that each pair should be on a line by itself, with the name and value separated by `=`.
3. Then write a second program that reads the data from the file, turning each line into a key-value pair in the dict.
4. Print the resulting dict.

Example:

    # my dict
    d ={'a':10, 'b':20, 'c':30}
    
    # after my code runs, I'll see on disk:
    a=10
    b=20
    c=30
    
    # then run the second program, and I get
    {'a':10, 'b':20, 'c':30}  # or maybe values are strings

In [7]:
# Part 1: Write a dict to disk

filename = 'reuven-config.txt'
d = {'a':10, 'b':20, 'c':30}

with open(filename, 'w') as f:          # open the file for writing, and get the "with" block ready
                                        # "as f" is the "with" way of saying f=
    for key, value in d.items():        # iterate over every key-value pair in d
        f.write(f'{key}={value}\n')     # write key=value to the file on a line by itself
                                        # the with block ends, the file is flushed + closed

In [8]:
!cat reuven-config.txt

a=10
b=20
c=30


In [9]:
# try this without "with"


filename = 'reuven-config.txt'
d = {'a':10, 'b':20, 'c':30}

f = open(filename, 'w')
for key, value in d.items():        
    f.write(f'{key}={value}\n')     
f.close()   # flush + close the file                                    

# Iterating over a dict with `dict.items`

If we iterate over a dictionary, we get just the keys.

Instead of doing that, we'll invoke `dict.items` and iterate over its results. With each iteration, we'll get a 2-element tuple of `(key, value)`.



In [11]:
# this will give me, one by one, each of the key-value pairs

for something in d.items():
    print(something)

('a', 10)
('b', 20)
('c', 30)


In [12]:
# Python allows us to grab the keys and values separately, using unpacking

for key, value in d.items():   # this works so long as d.items() always returns a 2-element tuple
    print(f'{key}: {value}')

a: 10
b: 20
c: 30


In [13]:
f = open(filename, 'a')
f.write('hello=123\n')

10

In [14]:
f.close()

In [15]:
!cat $filename

a=10
b=20
c=30
hello=123


In [16]:
for apple, banana in d.items():   # this works so long as d.items() always returns a 2-element tuple
    print(f'{apple}: {banana}')

a: 10
b: 20
c: 30


In [20]:
# Reading the data

newdict = {}

for one_line in open(filename):
    new_key, new_value = one_line.split('=')
    newdict[new_key] = new_value.strip()
    
newdict    

{'a': '10', 'b': '20', 'c': '30'}

# Functions

The "DRY rule," or "don't repeat yourself," means: Don't have the same code in more than one place.

- If the same code is on several lines in a row, then we can (should) use a loop.
- If the same code is in several different places in your program, then you can use a *function*.

Functions also give us semantic power -- we can wrap up a set of things we want to do, and put them under a single name, and then refer to that whole set of tasks as one name. This is known as "abstraction," and it's a crucial idea in programming.

Functions are verbs in programming, and if we can define a function, then we can define a new verb, and enjoy the advantage of the higher level of abstraction that it gives us.

# Defining a function

I can define a new function -- that is, teach Python a new word -- using the `def` keyword:

- `def`
- followed by the name that we want to give to a function
- followed by parentheses with any parameters the function has (at first, they will be empty)
- followed by `:`
- then an indented block, known as the "function body." This can be as long or short as you want, and it can contain any code you want.

In [24]:
def hello():           # indicate the name of the function, and any parameters it might have
    print('Hello!')    # the function body -- what happens when we run the function?

I just did two things:

- created a new function object in the system
- assigned it to the name `hello` -- yes, `def` is like `=`, in that it's assigning values to variables!

This means that you cannot, in Python, have both a function and a variable named the same thing. The one that was defined/assigned later wins.

In [22]:
type(hello)

function

In [23]:
hello()  # here's how I run the function -- with ()

Hello!


In [25]:
hello()

Hello!


In [26]:
for i in range(5):
    hello()

Hello!
Hello!
Hello!
Hello!
Hello!


In [27]:
def hello():
    name = input('Enter your name: ').strip()
    print(f'Hello, {name}!')

In [28]:
hello()

Enter your name: Reuven
Hello, Reuven!


In [32]:
def count_vowels():
    text = input('Enter text: ').strip()
    count = 0
    for one_character in text:
        if one_character in 'aeiou':
            count += 1
    print(f'Number of vowels: {count}')

In [30]:
count_vowels()

Enter tex: you forgot the t at the end of the sentence
Number of vowels: 13


# Exercise: Calculator

1. Define a function, `calc`, that takes no arguments / has no parameters.
2. Inside of the function, ask the user to enter a number, an operator, and another number. (Three different things, assigned to three different variables.)
3. The operator can be either `+` or `-`.
4. Print the result of the math operation they've requested.

Example:

    calc()
    Enter first number: 10
    Enter operator: +
    Enter second number: 5
    10 + 5 = 15
    
    calc()
    Enter first number: 10
    Enter operator: -
    Enter second number: 3
    10 - 3 = 7
    
Do this in PyCharm!    
